#### Import und Data Preparation der Kriminalstatistik

In [1]:
import pandas as pd
import numpy as np

In [2]:
url = "https://raw.githubusercontent.com/andreashrb/Python_for_DS_Project/main/data/Kriminalstatistik_Straftaten_2020.csv"

df_criminal = pd.read_csv(url,encoding="latin-1",skiprows=1,delimiter=";",usecols=range(18),decimal='.',thousands=',')
df_criminal.rename(columns={'Gemeindeschluessel':'Kreis_ID',
                           'Straftat':'Art_Straftat'},inplace=True)
df_criminal.drop(['Kreisart','Schluessel'],axis=1, inplace=True)


df_criminal.head()

,Art_Straftat,Kreis_ID,Stadt-/Landkreis,Anzahl erfasste Faelle,Haeufigkeitszahl,"erfasste Faelle, davon Versuche - Anzahl","erfasste Faelle, davon Versuche - Anteil in %",mit Schusswaffe gedroht,mit Schusswaffe geschossen,Auffklaerung - Anzahl Faelle,Aufklaerungsquote,Tatverdaechtige insgesamt,Tatverdaechtige - maennlich,Tatverdaechtige - weiblich,Nichtdeutsche Tatverdaechtige - Anzahl,Nichtdeutsche Tatverdaechtige - Anteil in % an der Gesamtzahl
0,Straftaten insgesamt,1001,Flensburg,8380,9294,598,7.1,11,5,5293,63.2,3618,2786,832,1233,34.1
1,Straftaten insgesamt,1002,Kiel,21384,8665,1413,6.6,23,5,11241,52.6,7278,5616,1662,2072,28.5
2,Straftaten insgesamt,1003,Lübeck,20942,9672,1513,7.2,26,13,11959,57.1,7603,5762,1841,1841,24.2
3,Straftaten insgesamt,1004,Neumünster,9453,11787,672,7.1,7,4,5836,61.7,4170,3097,1073,1955,46.9
4,Straftaten insgesamt,1051,Dithmarschen,6597,4953,457,6.9,13,12,3944,59.8,2989,2231,758,535,17.9


In [3]:
# Import und join der straftat_cluster

url_cluster = "https://raw.githubusercontent.com/andreashrb/Python_for_DS_Project/main/data/straftat_cluster.csv"

df_cluster = pd.read_csv(url_cluster, encoding="utf-8",delimiter=";")
df_cluster.rename(columns={'Straftatcluster':'Straftat_Cluster'},inplace=True)

df_criminal=pd.merge(df_criminal, df_cluster, left_on='Art_Straftat',right_on='Art_Straftat')


In [4]:
df_criminal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16842 entries, 0 to 16841
Data columns (total 17 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Art_Straftat                                                   16842 non-null  object 
 1   Kreis_ID                                                       16842 non-null  int64  
 2   Stadt-/Landkreis                                               16842 non-null  object 
 3   Anzahl erfasste Faelle                                         16842 non-null  int64  
 4   Haeufigkeitszahl                                               16842 non-null  int64  
 5   erfasste Faelle, davon Versuche - Anzahl                       16842 non-null  int64  
 6   erfasste Faelle, davon Versuche - Anteil in %                  16842 non-null  float64
 7   mit Schusswaffe gedroht                                   

In [5]:
# Löschen der 'Insgesamt' und 'und zwar' Datensätze, da diese bereits aggregierte Daten enthalten

df_criminal = df_criminal[df_criminal['Art_Straftat'].str.contains('insgesamt')==False]
df_criminal = df_criminal[df_criminal['Art_Straftat'].str.contains('und zwar')==False]

In [6]:
# Löschen der drei relativen Spalten, um Group by auf Summe durchführen zu können.

df_criminal.drop(['erfasste Faelle, davon Versuche - Anteil in %','Aufklaerungsquote','Nichtdeutsche Tatverdaechtige - Anteil in % an der Gesamtzahl'],axis=1, inplace=True)

In [7]:
df_criminal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12431 entries, 401 to 16841
Data columns (total 14 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Art_Straftat                              12431 non-null  object
 1   Kreis_ID                                  12431 non-null  int64 
 2   Stadt-/Landkreis                          12431 non-null  object
 3   Anzahl erfasste Faelle                    12431 non-null  int64 
 4   Haeufigkeitszahl                          12431 non-null  int64 
 5   erfasste Faelle, davon Versuche - Anzahl  12431 non-null  int64 
 6   mit Schusswaffe gedroht                   12431 non-null  int64 
 7   mit Schusswaffe geschossen                12431 non-null  int64 
 8   Auffklaerung - Anzahl Faelle              12431 non-null  int64 
 9   Tatverdaechtige insgesamt                 12431 non-null  int64 
 10  Tatverdaechtige - maennlich               12

In [8]:
# Group by auf Spalten Kreis_ID, Straftatcluster, Stadt-/Landkreis

df_criminal_grouped = df_criminal.groupby(['Kreis_ID','Straftat_Cluster','Stadt-/Landkreis']).sum()

df_criminal_grouped.head()

Anzahl erfasste Faelle  \
Kreis_ID Straftat_Cluster                                   Stadt-/Landkreis                           
1001     Beförderungserschleichung                          Flensburg                             57   
         Begünstigung, Strafvereitelung, Hehlerei, Geldw... Flensburg                             25   
         Betrug                                             Flensburg                            797   
         Brandstiftung                                      Flensburg                             28   
         Cyberkriminalität                                  Flensburg                            172   

                                                                              Haeufigkeitszahl  \
Kreis_ID Straftat_Cluster                                   Stadt-/Landkreis                     
1001     Beförderungserschleichung                          Flensburg                       63   
         Begünstigung, Strafvereitelung, Hehlerei, Geldw... Flensburg                       28   
         Betrug                                             Flensburg                      884   
         Brandstiftung                                      Flensburg                       31   
         Cyberkriminalität                                  Flensburg                      191   

                                                                              erfasste Faelle, davon Versuche - Anzahl  \
Kreis_ID Straftat_Cluster                                   Stadt-/Landkreis                                             
1001     Beförderungserschleichung                          Flensburg                                                0   
         Begünstigung, Strafvereitelung, Hehlerei, Geldw... Flensburg                                                0   
         Betrug                                             Flensburg                                               74   
         Brandstiftung                                      Flensburg                                                4   
         Cyberkriminalität                                  Flensburg                                               36   

                                                                              mit Schusswaffe gedroht  \
Kreis_ID Straftat_Cluster                                   Stadt-/Landkreis                            
1001     Beförderungserschleichung                          Flensburg                               0   
         Begünstigung, Strafvereitelung, Hehlerei, Geldw... Flensburg                               0   
         Betrug                                             Flensburg                               0   
         Brandstiftung                                      Flensburg                               0   
         Cyberkriminalität                                  Flensburg                               0   

                                                                              mit Schusswaffe geschossen  \
Kreis_ID Straftat_Cluster                                   Stadt-/Landkreis                               
1001     Beförderungserschleichung                          Flensburg                                  0   
         Begünstigung, Strafvereitelung, Hehlerei, Geldw... Flensburg                                  0   
         Betrug                                             Flensburg                                  0   
         Brandstiftung                                      Flensburg                                  0   
         Cyberkriminalität                                  Flensburg                                  0   

                                                                              Auffklaerung - Anzahl Faelle  \
Kreis_ID Straftat_Cluster                                   Stadt-/Landkreis                                 
1001     Beförderungserschleichung                          Flensburg                         

In [9]:
df_criminal_grouped = df_criminal_grouped.reset_index(drop=False)
df_criminal_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8822 entries, 0 to 8821
Data columns (total 13 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Kreis_ID                                  8822 non-null   int64 
 1   Straftat_Cluster                          8822 non-null   object
 2   Stadt-/Landkreis                          8822 non-null   object
 3   Anzahl erfasste Faelle                    8822 non-null   int64 
 4   Haeufigkeitszahl                          8822 non-null   int64 
 5   erfasste Faelle, davon Versuche - Anzahl  8822 non-null   int64 
 6   mit Schusswaffe gedroht                   8822 non-null   int64 
 7   mit Schusswaffe geschossen                8822 non-null   int64 
 8   Auffklaerung - Anzahl Faelle              8822 non-null   int64 
 9   Tatverdaechtige insgesamt                 8822 non-null   int64 
 10  Tatverdaechtige - maennlich               8822 n

In [10]:
# Neuberechnung der relativen Spalten

df_criminal_grouped['erfasste Faelle, davon Versuche - Anteil in %'] = np.where(df_criminal_grouped["erfasste Faelle, davon Versuche - Anzahl"] == 0, 0.00, (df_criminal_grouped["erfasste Faelle, davon Versuche - Anzahl"] / df_criminal_grouped["Anzahl erfasste Faelle"])*100)
# df_criminal_grouped['erfasste Faelle, davon Versuche - Anteil in %']=(df_criminal_grouped["erfasste Faelle, davon Versuche - Anzahl"] / df_criminal_grouped["Anzahl erfasste Faelle"])*100

df_criminal_grouped['Aufklaerungsquote in %'] = np.where(df_criminal_grouped['Auffklaerung - Anzahl Faelle'] == 0, 0.00, (df_criminal_grouped['Auffklaerung - Anzahl Faelle'] / df_criminal_grouped["Anzahl erfasste Faelle"])*100)
# df_criminal_grouped['Aufklaerungsquote in %']=(df_criminal_grouped['Auffklaerung - Anzahl Faelle'] / df_criminal_grouped["Anzahl erfasste Faelle"])*100

df_criminal_grouped['Nichtdeutsche Tatverdaechtige - Anteil in % an der Gesamtzahl'] = np.where(df_criminal_grouped['Nichtdeutsche Tatverdaechtige - Anzahl'] == 0, 0.00, (df_criminal_grouped['Nichtdeutsche Tatverdaechtige - Anzahl'] / df_criminal_grouped['Tatverdaechtige insgesamt'])*100)
# df_criminal_grouped['Nichtdeutsche Tatverdaechtige - Anteil in % an der Gesamtzahl']=(df_criminal_grouped['Nichtdeutsche Tatverdaechtige - Anzahl'] / df_criminal_grouped['Tatverdaechtige insgesamt'])*100

In [11]:
df_criminal_grouped.head()

,Kreis_ID,Straftat_Cluster,Stadt-/Landkreis,Anzahl erfasste Faelle,Haeufigkeitszahl,"erfasste Faelle, davon Versuche - Anzahl",mit Schusswaffe gedroht,mit Schusswaffe geschossen,Auffklaerung - Anzahl Faelle,Tatverdaechtige insgesamt,Tatverdaechtige - maennlich,Tatverdaechtige - weiblich,Nichtdeutsche Tatverdaechtige - Anzahl,"erfasste Faelle, davon Versuche - Anteil in %",Aufklaerungsquote in %,Nichtdeutsche Tatverdaechtige - Anteil in % an der Gesamtzahl
0,1001,Beförderungserschleichung,Flensburg,57,63,0,0,0,56,56,37,19,34,0.000000,98.245614,60.714286
1,1001,"Begünstigung, Strafvereitelung, Hehlerei, Geld...",Flensburg,25,28,0,0,0,24,27,20,7,9,0.000000,96.000000,33.333333
2,1001,Betrug,Flensburg,797,884,74,0,0,585,442,294,148,107,9.284818,73.400251,24.208145
3,1001,Brandstiftung,Flensburg,28,31,4,0,0,12,12,8,4,2,14.285714,42.857143,16.666667
4,1001,Cyberkriminalität,Flensburg,172,191,36,0,0,84,60,43,17,17,20.930233,48.837209,28.333333


In [12]:
df_criminal_grouped[['Stadt-/Landkreis','Straftat_Cluster','Auffklaerung - Anzahl Faelle',"Anzahl erfasste Faelle",'Aufklaerungsquote in %']] [df_criminal_grouped['Straftat_Cluster']=='Diebstahl insgesamt']

,Stadt-/Landkreis,Straftat_Cluster,Auffklaerung - Anzahl Faelle,Anzahl erfasste Faelle,Aufklaerungsquote in %


In [13]:
df_criminal_grouped.isnull().sum().sort_values(ascending=False).head(20)

Kreis_ID                                                         0
Straftat_Cluster                                                 0
Stadt-/Landkreis                                                 0
Anzahl erfasste Faelle                                           0
Haeufigkeitszahl                                                 0
erfasste Faelle, davon Versuche - Anzahl                         0
mit Schusswaffe gedroht                                          0
mit Schusswaffe geschossen                                       0
Auffklaerung - Anzahl Faelle                                     0
Tatverdaechtige insgesamt                                        0
Tatverdaechtige - maennlich                                      0
Tatverdaechtige - weiblich                                       0
Nichtdeutsche Tatverdaechtige - Anzahl                           0
erfasste Faelle, davon Versuche - Anteil in %                    0
Aufklaerungsquote in %                                        

In [14]:
df_criminal = df_criminal_grouped

In [15]:
df_criminal.shape[0]

8822

#### Import von Daten zu beruflichen Absolventen

Bei diesem Datensatz handelt es sich um Informationen zur Anzahl der Absolventen bzw. Abgängern von beruflichen Schulen. Der Datensatz ist gegliedert nach Geschlecht und Abschlussart. Die Daten sind aus dem Abschlussjahrgang 2020 und sind folgender Quelle entnommen: https://www.regionalstatistik.de/genesis//online?operation=table&code=21121-02-02-4&bypass=true&levelindex=1&levelid=1645033448667#abreadcrumb

In [16]:
# Import der Daten aus dem github-repository

url_grad_prof_school = "https://raw.githubusercontent.com/andreashrb/Python_for_DS_Project/main/data/Absolventen_2020_berufliche_Schulen.csv"

df_graduates_prof_school = pd.read_csv(url_grad_prof_school,encoding="latin-1",skiprows=4,delimiter=";", decimal=",",header=[1,2,3,4,5,6])

df_graduates_prof_school.replace('-', None)

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                    Unnamed: 0_level_1 Unnamed: 1_level_1   
                                    Unnamed: 0_level_2 Unnamed: 1_level_2   
                                    Unnamed: 0_level_3 Unnamed: 1_level_3   
                                    Unnamed: 0_level_4 Unnamed: 1_level_4   
                                    Unnamed: 0_level_5 Unnamed: 1_level_5   
0                                                 2020                 DG   
1                                                 2020                 01   
2                                                 2020              01001   
3                                                 2020              01002   
4                                                 2020              01003   
..                                                 ...                ...   
537                                               2020              16077   
538                                         __________                NaN   
539  zu "gesamte Tabelle":\nDeutschland, Sachsen-An...                NaN   
540  © Statistische Ämter des Bundes und der Länder...                NaN   
541                       Stand: 13.02.2022 / 12:59:18                NaN   

                                 Unnamed: 2_level_0 Absolventen/Abgänger  \
                                 Unnamed: 2_level_1           Geschlecht   
                                 Unnamed: 2_level_2            Insgesamt   
                                 Unnamed: 2_level_3   Unnamed: 3_level_3   
                                 Unnamed: 2_level_4   Unnamed: 3_level_4   
                                 Unnamed: 2_level_5               Anzahl   
0                                       Deutschland               251232   
1                                Schleswig-Holstein                 9516   
2                       Flensburg, kreisfreie Stadt                  804   
3          Kiel, Landeshauptstadt, kreisfreie Stadt                 1281   
4              Lübeck, Hansestadt, kreisfreie Stadt                  960   
..                                              ...                  ...   
537                         Altenburger Land, Kreis                  149   
538                                             NaN                  NaN   
539                                             NaN                  NaN   
540                                             NaN                  NaN   
541                                             NaN                  NaN   

                                                         \
                                     davon mit            
              weiblich mit Hauptschulabschluss            
    Unnamed: 4_level_3              Geschlecht            
    Unnamed: 4_level_4               Insgesamt weiblich   
                Anzahl                  Anzahl   Anzahl   
0               120282                   48174    17949   
1                 4350                    2079      786   
2                  342                     189       69   
3                  576                     168       57   
4                  444                     150       60   
..                 ...                     ...      ...   
537                 78                      28        7   
538                NaN                     NaN      NaN   
539                NaN                     NaN      NaN   
540                NaN                     NaN      NaN   
541                NaN                     NaN      NaN   

                                                                  \
                                                                   
    mit Mittlerem Abschluss          Fachhochschulreife            
                 Geschlecht                  Geschlecht            
                  Insgesamt weiblich          Insgesamt weiblich   
                     Anzahl   Anzahl             Anzahl   Anzahl   
0                    

In [17]:
# Auflösen und Zusammenführen der mehrstufigen Spaltenstruktur sowie Umbenennung der Spalten

df_graduates_prof_school.columns = [' '.join(col) for col in df_graduates_prof_school]

df_graduates_prof_school.rename(columns={'Unnamed: 0_level_0 Unnamed: 0_level_1 Unnamed: 0_level_2 Unnamed: 0_level_3 Unnamed: 0_level_4 Unnamed: 0_level_5':'Jahr',
                                        'Unnamed: 1_level_0 Unnamed: 1_level_1 Unnamed: 1_level_2 Unnamed: 1_level_3 Unnamed: 1_level_4 Unnamed: 1_level_5':'Kreis_ID',
                                        'Unnamed: 2_level_0 Unnamed: 2_level_1 Unnamed: 2_level_2 Unnamed: 2_level_3 Unnamed: 2_level_4 Unnamed: 2_level_5':'Kreis/Stadt',
                                        'Absolventen/Abgänger Geschlecht Insgesamt Unnamed: 3_level_3 Unnamed: 3_level_4 Anzahl':'Berufl. Absolventen Anzahl insgesamt',
                                        'Absolventen/Abgänger Geschlecht weiblich Unnamed: 4_level_3 Unnamed: 4_level_4 Anzahl':'Berufl. Absolventen Anzahl weiblich',
                                        'Absolventen/Abgänger davon mit mit Hauptschulabschluss Geschlecht Insgesamt Anzahl':'Berufl. Absolventen Anzahl mit Hauptschulabschluss insgesamt',
                                        'Absolventen/Abgänger davon mit mit Hauptschulabschluss Geschlecht weiblich Anzahl':'Berufl. Absolventen Anzahl mit Hauptschulabschluss weiblich',
                                        'Absolventen/Abgänger davon mit mit Mittlerem Abschluss Geschlecht Insgesamt Anzahl':'Berufl. Absolventen Anzahl mit mittlerem Abschluss insgesamt',
                                        'Absolventen/Abgänger davon mit mit Mittlerem Abschluss Geschlecht weiblich Anzahl':'Berufl. Absolventen Anzahl mit mittlerem Abschluss weiblich',
                                        'Absolventen/Abgänger davon mit Fachhochschulreife Geschlecht Insgesamt Anzahl':'Berufl. Absolventen Anzahl mit Fachhochschulreife insgesamt',
                                        'Absolventen/Abgänger davon mit Fachhochschulreife Geschlecht weiblich Anzahl':'Berufl. Absolventen Anzahl mit Fachhochschulreife weiblich',
                                        'Absolventen/Abgänger davon mit Allg. Hochschulreife-einschl.fachge.Hochschulreife Geschlecht Insgesamt Anzahl':'Berufl. Absolventen Anzahl mit allg. Hochschulreife einschl. fachgeb. Hochschulreife insgesamt',
                                        'Absolventen/Abgänger davon mit Allg. Hochschulreife-einschl.fachge.Hochschulreife Geschlecht weiblich Anzahl':'Berufl. Absolventen Anzahl mit allg. Hochschulreife einschl. fachgeb. Hochschulreife weiblich'},inplace=True)

df_graduates_prof=df_graduates_prof_school[df_graduates_prof_school.columns[1:12]]

In [18]:
# Entfernen der vorangestellten 0 bei den vierstelligen Kreis_IDs, da für den Join nicht benötigt

for col in df_graduates_prof:
    df_graduates_prof['Kreis_ID'] = df_graduates_prof['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)
    
df_grad_prof = df_graduates_prof

df_grad_prof.head()

C:\Users\Andreas\AppData\Local\Temp/ipykernel_28820/1909248432.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graduates_prof['Kreis_ID'] = df_graduates_prof['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)


,Kreis_ID,Kreis/Stadt,Berufl. Absolventen Anzahl insgesamt,Berufl. Absolventen Anzahl weiblich,Berufl. Absolventen Anzahl mit Hauptschulabschluss insgesamt,Berufl. Absolventen Anzahl mit Hauptschulabschluss weiblich,Berufl. Absolventen Anzahl mit mittlerem Abschluss insgesamt,Berufl. Absolventen Anzahl mit mittlerem Abschluss weiblich,Berufl. Absolventen Anzahl mit Fachhochschulreife insgesamt,Berufl. Absolventen Anzahl mit Fachhochschulreife weiblich,Berufl. Absolventen Anzahl mit allg. Hochschulreife einschl. fachgeb. Hochschulreife insgesamt
0,DG,Deutschland,251232,120282,48174,17949,68715,32997,81423,40134,52923
1,1,Schleswig-Holstein,9516,4350,2079,786,2847,1296,1965,858,2625
2,1001,"Flensburg, kreisfreie Stadt",804,342,189,69,243,90,141,57,231
3,1002,"Kiel, Landeshauptstadt, kreisfreie Stadt",1281,576,168,57,348,141,312,138,456
4,1003,"Lübeck, Hansestadt, kreisfreie Stadt",960,444,150,60,309,150,231,84,270


In [19]:
# Bearbeitung von Hamburg und Berlin, um Join zu ermöglichen
# Spalte Kreis/Stadt löschen

df_grad_prof.loc[df_grad_prof['Kreis_ID']=='11','Kreis_ID']='11000'
df_grad_prof.loc[df_grad_prof['Kreis_ID']=='2','Kreis_ID']='2000'

#df_grad_prof[df_grad_prof["Kreis/Stadt"].str.contains("Hamburg")==True]
#df_grad_prof[df_grad_prof["Kreis/Stadt"].str.contains("Berlin")==True]



df_graduates_prof.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column                                                                                          Non-Null Count  Dtype 
---  ------                                                                                          --------------  ----- 
 0   Kreis_ID                                                                                        542 non-null    object
 1   Kreis/Stadt                                                                                     538 non-null    object
 2   Berufl. Absolventen Anzahl insgesamt                                                            538 non-null    object
 3   Berufl. Absolventen Anzahl weiblich                                                             538 non-null    object
 4   Berufl. Absolventen Anzahl mit Hauptschulabschluss insgesamt                                    538 non-null    object
 5   Berufl. Absolventen An

C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [20]:
# Auswahl aller Kreis_IDs mit Zeilenlänge > 4
# Umwandelung in Datentyp int

df_grad=df_graduates_prof[df_graduates_prof.apply(lambda x: len(x['Kreis_ID'])>=4, axis=1)]

df_grad=df_grad.replace('.',None)
df_grad=df_grad.replace('-',None)
df_grad=df_grad.drop(['Kreis/Stadt'], axis=1)
df_grad=df_grad.apply(pd.to_numeric)

df_grad.head()

,Kreis_ID,Berufl. Absolventen Anzahl insgesamt,Berufl. Absolventen Anzahl weiblich,Berufl. Absolventen Anzahl mit Hauptschulabschluss insgesamt,Berufl. Absolventen Anzahl mit Hauptschulabschluss weiblich,Berufl. Absolventen Anzahl mit mittlerem Abschluss insgesamt,Berufl. Absolventen Anzahl mit mittlerem Abschluss weiblich,Berufl. Absolventen Anzahl mit Fachhochschulreife insgesamt,Berufl. Absolventen Anzahl mit Fachhochschulreife weiblich,Berufl. Absolventen Anzahl mit allg. Hochschulreife einschl. fachgeb. Hochschulreife insgesamt
2,1001,804,342,189,69,243,90,141,57,231
3,1002,1281,576,168,57,348,141,312,138,456
4,1003,960,444,150,60,309,150,231,84,270
5,1004,666,297,165,66,183,78,159,69,162
6,1051,324,159,48,21,120,60,39,12,117


In [21]:
# Join an den Kriminaldatensatz

group_criminal=pd.merge(df_criminal, df_grad, on=['Kreis_ID'],how='left')
group_criminal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8822 entries, 0 to 8821
Data columns (total 25 columns):
 #   Column                                                                                          Non-Null Count  Dtype  
---  ------                                                                                          --------------  -----  
 0   Kreis_ID                                                                                        8822 non-null   int64  
 1   Straftat_Cluster                                                                                8822 non-null   object 
 2   Stadt-/Landkreis                                                                                8822 non-null   object 
 3   Anzahl erfasste Faelle                                                                          8822 non-null   int64  
 4   Haeufigkeitszahl                                                                                8822 non-null   int64  
 5   erfasste Fael

In [22]:
#Check auf missing values

group_criminal.isnull().sum(axis=0)

Kreis_ID                                                                                          0
Straftat_Cluster                                                                                  0
Stadt-/Landkreis                                                                                  0
Anzahl erfasste Faelle                                                                            0
Haeufigkeitszahl                                                                                  0
erfasste Faelle, davon Versuche - Anzahl                                                          0
mit Schusswaffe gedroht                                                                           0
mit Schusswaffe geschossen                                                                        0
Auffklaerung - Anzahl Faelle                                                                      0
Tatverdaechtige insgesamt                                                                         0


In [23]:
group_criminal.head()

,Kreis_ID,Straftat_Cluster,Stadt-/Landkreis,Anzahl erfasste Faelle,Haeufigkeitszahl,"erfasste Faelle, davon Versuche - Anzahl",mit Schusswaffe gedroht,mit Schusswaffe geschossen,Auffklaerung - Anzahl Faelle,Tatverdaechtige insgesamt,...,Nichtdeutsche Tatverdaechtige - Anteil in % an der Gesamtzahl,Berufl. Absolventen Anzahl insgesamt,Berufl. Absolventen Anzahl weiblich,Berufl. Absolventen Anzahl mit Hauptschulabschluss insgesamt,Berufl. Absolventen Anzahl mit Hauptschulabschluss weiblich,Berufl. Absolventen Anzahl mit mittlerem Abschluss insgesamt,Berufl. Absolventen Anzahl mit mittlerem Abschluss weiblich,Berufl. Absolventen Anzahl mit Fachhochschulreife insgesamt,Berufl. Absolventen Anzahl mit Fachhochschulreife weiblich,Berufl. Absolventen Anzahl mit allg. Hochschulreife einschl. fachgeb. Hochschulreife insgesamt
0,1001,Beförderungserschleichung,Flensburg,57,63,0,0,0,56,56,...,60.714286,804,342,189,69,243,90,141,57,231
1,1001,"Begünstigung, Strafvereitelung, Hehlerei, Geld...",Flensburg,25,28,0,0,0,24,27,...,33.333333,804,342,189,69,243,90,141,57,231
2,1001,Betrug,Flensburg,797,884,74,0,0,585,442,...,24.208145,804,342,189,69,243,90,141,57,231
3,1001,Brandstiftung,Flensburg,28,31,4,0,0,12,12,...,16.666667,804,342,189,69,243,90,141,57,231
4,1001,Cyberkriminalität,Flensburg,172,191,36,0,0,84,60,...,28.333333,804,342,189,69,243,90,141,57,231


#### Import von Daten zu schulischen Absolventen

Bei diesem Datensatz handelt es sich um Informationen zur Anzahl der Absolventen bzw. Abgängern von allgemeinbildenden Schulen. Der Datensatz ist gegliedert nach Geschlecht und Abschlussart. Die Daten sind aus dem Abschlussjahrgang 2020 und sind folgender Quelle entnommen: https://www.regionalstatistik.de/genesis//online?operation=table&code=21111-02-06-4&bypass=true&levelindex=1&levelid=1645036254030#abreadcrumb

In [24]:
# Import der Daten aus dem github-repository

url_grad_gen_school = 'https://raw.githubusercontent.com/andreashrb/Python_for_DS_Project/main/data/Absolventen_2020_allgemeinbildende%20Schulen.csv'

df_graduates_gen_school = pd.read_csv(url_grad_gen_school,encoding="latin-1",skiprows=4,delimiter=";", decimal=",",header=[1,2,3,4])

df_graduates_gen_school.replace('-', None)

,Absolventen/Abgänger (Anzahl),Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,Unnamed: 10_level_0,Unnamed: 11_level_0,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0,Unnamed: 16_level_0
,Unnamed: 0_level_1,Unnamed: 1_level_1,Unnamed: 2_level_1,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss,Schulabschluss
,Unnamed: 0_level_2,Unnamed: 1_level_2,Unnamed: 2_level_2,Insgesamt,Insgesamt,ohne Hauptschulabschluss,ohne Hauptschulabschluss,mit Hauptschulabschluss,mit Hauptschulabschluss,mit Mittlerem Abschluss,mit Mittlerem Abschluss,dar. schulischer Teil der Fachhochschulreife,dar. schulischer Teil der Fachhochschulreife,mit Fachhochschulreife,mit Fachhochschulreife,mit allgemeiner Hochschulreife,mit allgemeiner Hochschulreife
,Unnamed: 0_level_3,Unnamed: 1_level_3,Unnamed: 2_level_3,Insgesamt,weiblich,Insgesamt,weiblich,Insgesamt,weiblich,Insgesamt,weiblich,Insgesamt,weiblich,Insgesamt,weiblich,Insgesamt,weiblich
0,2020,DG,Deutschland,749946,363864,45072,17139,123687,49083,333039,161535,17649,8439,912,441,247236,135669
1,2020,01,Schleswig-Holstein,27585,13407,2283,858,5010,2010,10272,5121,1131,549,912,441,10023,5418
2,2020,01001,"Flensburg, kreisfreie Stadt",1116,555,84,39,147,57,393,204,60,33,912,441,489,258
3,2020,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",2136,1050,186,75,357,144,669,303,81,33,912,441,924,528
4,2020,01003,"Lübeck, Hansestadt, kreisfreie Stadt",1851,891,192,75,327,117,612,300,66,30,912,441,720,399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2020,16077,"Altenburger Land, Kreis",768,363,58,21,117,48,347,164,8,6,x,x,246,130


In [25]:
# Auflösen und Zusammenführen der mehrstufigen Spaltenstruktur sowie Umbenennung der Spalten

df_graduates_gen_school.columns = [' '.join(col) for col in df_graduates_gen_school]
df_graduates_gen_school

df_graduates_gen_school.rename(columns={'Absolventen/Abgänger (Anzahl) Unnamed: 0_level_1 Unnamed: 0_level_2 Unnamed: 0_level_3':'Jahr',
                                       'Unnamed: 1_level_0 Unnamed: 1_level_1 Unnamed: 1_level_2 Unnamed: 1_level_3':'Kreis_ID',
                                       'Unnamed: 2_level_0 Unnamed: 2_level_1 Unnamed: 2_level_2 Unnamed: 2_level_3':'Kreis/Stadt',
                                       'Unnamed: 3_level_0 Schulabschluss Insgesamt Insgesamt':'Allg. Absolventen Anzahl insgesamt',
                                       'Unnamed: 4_level_0 Schulabschluss Insgesamt weiblich':'Allg. Absolventen Anzahl weiblich',
                                       'Unnamed: 5_level_0 Schulabschluss ohne Hauptschulabschluss Insgesamt':'Allg. Absolventen Anzahl ohne Hauptschulabschluss Insgesamt',
                                       'Unnamed: 6_level_0 Schulabschluss ohne Hauptschulabschluss weiblich':'Allg. Absolventen Anzahl ohne Hauptschulabschluss weiblich',
                                       'Unnamed: 7_level_0 Schulabschluss mit Hauptschulabschluss Insgesamt':'Allg. Absolventen Anzahl mit Hauptschulabschluss Insgesamt',
                                       'Unnamed: 8_level_0 Schulabschluss mit Hauptschulabschluss weiblich':'Allg. Absolventen Anzahl mit Hauptschulabschluss weiblich',
                                       'Unnamed: 9_level_0 Schulabschluss mit Mittlerem Abschluss Insgesamt':'Allg. Absolventen Anzahl mit mittlerer Reife Insgesamt',
                                       'Unnamed: 10_level_0 Schulabschluss mit Mittlerem Abschluss weiblich':'Allg. Absolventen Anzahl mit mittlerer Reife weiblich',
                                       'Unnamed: 11_level_0 Schulabschluss dar. schulischer Teil der Fachhochschulreife Insgesamt':'Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife Insgesamt',
                                       'Unnamed: 12_level_0 Schulabschluss dar. schulischer Teil der Fachhochschulreife weiblich': 'Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife weiblich',
                                       'Unnamed: 13_level_0 Schulabschluss mit Fachhochschulreife Insgesamt':'Allg. Absolventen Anzahl mit Fachhochschulreife Insgesamt',
                                       'Unnamed: 14_level_0 Schulabschluss mit Fachhochschulreife weiblich':'Allg. Absolventen Anzahl mit Fachhochschulreife weiblich',
                                       'Unnamed: 15_level_0 Schulabschluss mit allgemeiner Hochschulreife Insgesamt':'Allg. Absolventen Anzahl mit allg. Hochschulreife Insgesamt',
                                       'Unnamed: 16_level_0 Schulabschluss mit allgemeiner Hochschulreife weiblich':'Allg. Absolventen Anzahl mit allg. Hochschulreife weiblich'},inplace=True)

df_grad_gen=df_graduates_gen_school[df_graduates_gen_school.columns[1:12]]

In [26]:
df_grad_gen.head()

,Kreis_ID,Kreis/Stadt,Allg. Absolventen Anzahl insgesamt,Allg. Absolventen Anzahl weiblich,Allg. Absolventen Anzahl ohne Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl ohne Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl mit Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit mittlerer Reife Insgesamt,Allg. Absolventen Anzahl mit mittlerer Reife weiblich,Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife Insgesamt
0,DG,Deutschland,749946,363864,45072,17139,123687,49083,333039,161535,17649
1,01,Schleswig-Holstein,27585,13407,2283,858,5010,2010,10272,5121,1131
2,01001,"Flensburg, kreisfreie Stadt",1116,555,84,39,147,57,393,204,60
3,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",2136,1050,186,75,357,144,669,303,81
4,01003,"Lübeck, Hansestadt, kreisfreie Stadt",1851,891,192,75,327,117,612,300,66


In [27]:
# Entfernen der vorangestellten 0 bei den vierstelligen Kreis_IDs, da für den Join nicht benötigt

for co in df_grad_gen:
    df_grad_gen['Kreis_ID'] = df_grad_gen['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)

df_grad_gen

C:\Users\Andreas\AppData\Local\Temp/ipykernel_28820/3496966611.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grad_gen['Kreis_ID'] = df_grad_gen['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)


,Kreis_ID,Kreis/Stadt,Allg. Absolventen Anzahl insgesamt,Allg. Absolventen Anzahl weiblich,Allg. Absolventen Anzahl ohne Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl ohne Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl mit Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit mittlerer Reife Insgesamt,Allg. Absolventen Anzahl mit mittlerer Reife weiblich,Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife Insgesamt
0,DG,Deutschland,749946,363864,45072,17139,123687,49083,333039,161535,17649
1,1,Schleswig-Holstein,27585,13407,2283,858,5010,2010,10272,5121,1131
2,1001,"Flensburg, kreisfreie Stadt",1116,555,84,39,147,57,393,204,60
3,1002,"Kiel, Landeshauptstadt, kreisfreie Stadt",2136,1050,186,75,357,144,669,303,81
4,1003,"Lübeck, Hansestadt, kreisfreie Stadt",1851,891,192,75,327,117,612,300,66
...,...,...,...,...,...,...,...,...,...,...,...
537,16077,"Altenburger Land, Kreis",768,363,58,21,117,48,347,164,8
538,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Bearbeitung von Hamburg und Berlin, um Join zu ermöglichen
# Spalte Kreis/Stadt löschen

df_grad_gen.loc[df_grad_gen['Kreis_ID']=='11','Kreis_ID']='11000'
df_grad_gen.loc[df_grad_gen['Kreis_ID']=='2','Kreis_ID']='2000'

#df_grad_gen[df_grad_gen["Kreis/Stadt"].str.contains("Hamburg")==True]
#df_grad_gen[df_grad_gen["Kreis/Stadt"].str.contains("Berlin")==True]



df_grad_gen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column                                                                          Non-Null Count  Dtype 
---  ------                                                                          --------------  ----- 
 0   Kreis_ID                                                                        542 non-null    object
 1   Kreis/Stadt                                                                     538 non-null    object
 2   Allg. Absolventen Anzahl insgesamt                                              538 non-null    object
 3   Allg. Absolventen Anzahl weiblich                                               538 non-null    object
 4   Allg. Absolventen Anzahl ohne Hauptschulabschluss Insgesamt                     538 non-null    object
 5   Allg. Absolventen Anzahl ohne Hauptschulabschluss weiblich                      538 non-null    object
 6   Allg. Absolventen Anzahl m

C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [29]:
# Auswahl aller Kreis_IDs mit Zeilenlänge > 4
# Umwandelung in Datentyp int

df_grad_gen=df_grad_gen[df_grad_gen.apply(lambda x: len(x['Kreis_ID'])>=4, axis=1)]

df_grad_gen=df_grad_gen.replace('.',None)
df_grad_gen=df_grad_gen.replace('-',None)
df_grad_gen=df_grad_gen.replace('x',None)
df_grad_gen=df_grad_gen.drop(['Kreis/Stadt'], axis=1)
df_grad_gen=df_grad_gen.apply(pd.to_numeric)

df_grad_gen.head()

,Kreis_ID,Allg. Absolventen Anzahl insgesamt,Allg. Absolventen Anzahl weiblich,Allg. Absolventen Anzahl ohne Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl ohne Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl mit Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit mittlerer Reife Insgesamt,Allg. Absolventen Anzahl mit mittlerer Reife weiblich,Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife Insgesamt
2,1001,1116,555,84,39,147,57,393,204,60
3,1002,2136,1050,186,75,357,144,669,303,81
4,1003,1851,891,192,75,327,117,612,300,66
5,1004,1038,519,75,30,204,87,267,132,60
6,1051,1338,630,135,39,270,93,594,309,63


In [30]:
# Join an den Kriminaldatensatz

group_criminal1=pd.merge(group_criminal, df_grad_gen, on=['Kreis_ID'],how='left')
group_criminal1.head()

,Kreis_ID,Straftat_Cluster,Stadt-/Landkreis,Anzahl erfasste Faelle,Haeufigkeitszahl,"erfasste Faelle, davon Versuche - Anzahl",mit Schusswaffe gedroht,mit Schusswaffe geschossen,Auffklaerung - Anzahl Faelle,Tatverdaechtige insgesamt,...,Berufl. Absolventen Anzahl mit allg. Hochschulreife einschl. fachgeb. Hochschulreife insgesamt,Allg. Absolventen Anzahl insgesamt,Allg. Absolventen Anzahl weiblich,Allg. Absolventen Anzahl ohne Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl ohne Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit Hauptschulabschluss Insgesamt,Allg. Absolventen Anzahl mit Hauptschulabschluss weiblich,Allg. Absolventen Anzahl mit mittlerer Reife Insgesamt,Allg. Absolventen Anzahl mit mittlerer Reife weiblich,Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife Insgesamt
0,1001,Beförderungserschleichung,Flensburg,57,63,0,0,0,56,56,...,231,1116,555,84,39,147,57,393,204,60
1,1001,"Begünstigung, Strafvereitelung, Hehlerei, Geld...",Flensburg,25,28,0,0,0,24,27,...,231,1116,555,84,39,147,57,393,204,60
2,1001,Betrug,Flensburg,797,884,74,0,0,585,442,...,231,1116,555,84,39,147,57,393,204,60
3,1001,Brandstiftung,Flensburg,28,31,4,0,0,12,12,...,231,1116,555,84,39,147,57,393,204,60
4,1001,Cyberkriminalität,Flensburg,172,191,36,0,0,84,60,...,231,1116,555,84,39,147,57,393,204,60


In [31]:
#Check auf missing values

group_criminal1.isnull().sum(axis=0)

Kreis_ID                                                                                          0
Straftat_Cluster                                                                                  0
Stadt-/Landkreis                                                                                  0
Anzahl erfasste Faelle                                                                            0
Haeufigkeitszahl                                                                                  0
erfasste Faelle, davon Versuche - Anzahl                                                          0
mit Schusswaffe gedroht                                                                           0
mit Schusswaffe geschossen                                                                        0
Auffklaerung - Anzahl Faelle                                                                      0
Tatverdaechtige insgesamt                                                                         0


#### Import von Daten zu Arbeitslosen im Jahr 2020

Bei diesem Datensatz handelt es sich um einen Extrakt aus statistischen Daten der Arbeitslosenstatistik. Es sind Informationen zur Anzahl von Arbeitslosen nach ausgewählten Personengruppen im Jahresdurchschnitt sowie die entsprechenden Quoten vorhanden. Der Datensatz ist gegliedert nach Altersgruppen (15 bis unter 20 Jahre, 15 bis unter 25 Jahre sowie 55 bis unter 65 Jahre). Außerdem sind die Anzahl der arbeitslosen Ausländer, arbeitslosen Schwerbehinderten und die Langzeitarbeitslosen enthalten. Die Daten sind aus dem Jahr 2020 und sind folgender Quelle entnommen:
https://www.regionalstatistik.de/genesis/online#astructure

In [32]:
# Import der Daten aus dem github-repository

url_unemployed = 'https://raw.githubusercontent.com/andreashrb/Python_for_DS_Project/main/data/Arbeitslose_Arbeitslosenquote_Jahr_2020_nach_ausgew%C3%A4hlten_Personengruppen.csv'

df_unemployed = pd.read_csv(url_unemployed,encoding="latin-1",skiprows=3,delimiter=";", decimal=",",header=[1,2,3,4],na_values=['(NA)']).fillna(0)

df_unemployed.replace('-', None)

,Arbeitsmarktstatistik der Bundesagentur für Arbeit,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,Unnamed: 10_level_0,Unnamed: 11_level_0,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0
,Unnamed: 0_level_1,Unnamed: 1_level_1,Unnamed: 2_level_1,Arbeitslose,Arbeitslose,Arbeitslose,Arbeitslose,Arbeitslose,Arbeitslose,Arbeitslose,Arbeitslosenquote bez. abh. zivile Erwerbspers.,Arbeitslosenquote bez. auf alle zivile Erwerbsp.,Arbeitslosenquote bez. auf alle zivile Erwerbsp.,Arbeitslosenquote bez. auf alle zivile Erwerbsp.,Arbeitslosenquote bez. auf alle zivile Erwerbsp.,Arbeitslosenquote bez. auf alle zivile Erwerbsp.
,Unnamed: 0_level_2,Unnamed: 1_level_2,Unnamed: 2_level_2,Unnamed: 3_level_2,Ausländer,schwerbehindert,15 bis unter 20 Jahre,15 bis unter 25 Jahre,55 bis unter 65 Jahre,langzeitarbeitslos,Unnamed: 10_level_2,Unnamed: 11_level_2,Männer,Frauen,Ausländer,15 bis unter 25 Jahre
,Unnamed: 0_level_3,Unnamed: 1_level_3,Unnamed: 2_level_3,Anzahl,Anzahl,Anzahl,Anzahl,Anzahl,Anzahl,Anzahl,Prozent,Prozent,Prozent,Prozent,Prozent,Prozent
0,2020,DG,Deutschland,2695444,795692,169691,49477,257361,562573,816749,"6,5","5,9","6,3","5,5","14,4","5,5"
1,2020,01,Schleswig-Holstein,92140,21506,5197,1842,9308,19359,28170,"6,5","5,8","6,3","5,3","18,8","5,5"
2,2020,01001,"Flensburg, kreisfreie Stadt",4722,1144,274,102,568,782,1419,"9,9","9,1","10,3","7,7","19,4","7,6"
3,2020,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",11776,3415,535,183,931,1959,4159,"9,3","8,5","9,4","7,4","24,9","5,4"
4,2020,01003,"Lübeck, Hansestadt, kreisfreie Stadt",9404,2315,477,190,922,1746,3131,"9,0","8,1","8,9","7,3","21,3","7,4"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2020,16077,"Altenburger Land, Kreis",3176,368,196,67,271,864,1168,"7,7","7,1","7,6","6,5","25,9","8,9"


In [33]:
# Auflösen und Zusammenführen der mehrstufigen Spaltenstruktur sowie Umbenennung der Spalten

df_unemployed.columns = [' '.join(col) for col in df_unemployed]

df_unemployed.rename(columns={'Arbeitsmarktstatistik der Bundesagentur für Arbeit Unnamed: 0_level_1 Unnamed: 0_level_2 Unnamed: 0_level_3':'Jahr',
                                        'Unnamed: 1_level_0 Unnamed: 1_level_1 Unnamed: 1_level_2 Unnamed: 1_level_3':'Kreis_ID',
                                        'Unnamed: 2_level_0 Unnamed: 2_level_1 Unnamed: 2_level_2 Unnamed: 2_level_3':'Kreis/Stadt',
                                        'Unnamed: 3_level_0 Arbeitslose Unnamed: 3_level_2 Anzahl':'Arbeitslose Anzahl insgesamt',
                                        'Unnamed: 4_level_0 Arbeitslose Ausländer Anzahl':'Arbeitslose Anzahl Ausländer',
                                        'Unnamed: 5_level_0 Arbeitslose schwerbehindert Anzahl':'Arbeitslose Anzahl Schwerbehindert',
                                        'Unnamed: 6_level_0 Arbeitslose 15 bis unter 20 Jahre Anzahl':'Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre',
                                        'Unnamed: 7_level_0 Arbeitslose 15 bis unter 25 Jahre Anzahl':'Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre',
                                        'Unnamed: 8_level_0 Arbeitslose 55 bis unter 65 Jahre Anzahl':'Arbeitslose Anzahl Alter zwischen 55 bis unter 65 Jahre',
                                        'Unnamed: 9_level_0 Arbeitslose langzeitarbeitslos Anzahl':'Arbeitslose Anzahl Langzeitarbeitslos',
                                        'Unnamed: 10_level_0 Arbeitslosenquote bez. abh. zivile Erwerbspers. Unnamed: 10_level_2 Prozent':'Arbeitslosenquote Prozent bezogen auf abhängige zivile Erwerbspersonen',
                                        'Unnamed: 11_level_0 Arbeitslosenquote bez. auf alle zivile Erwerbsp. Unnamed: 11_level_2 Prozent':'Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen',
                                        'Unnamed: 12_level_0 Arbeitslosenquote bez. auf alle zivile Erwerbsp. Männer Prozent':'Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen - männlich',
                                        'Unnamed: 13_level_0 Arbeitslosenquote bez. auf alle zivile Erwerbsp. Frauen Prozent':'Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen - weiblich',
                                        'Unnamed: 14_level_0 Arbeitslosenquote bez. auf alle zivile Erwerbsp. Ausländer Prozent':'Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen - Ausländer',
                                        'Unnamed: 15_level_0 Arbeitslosenquote bez. auf alle zivile Erwerbsp. 15 bis unter 25 Jahre Prozent':'Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen - zwischen 15 bis unter 25 Jahre'},inplace=True)

df_unem=df_unemployed[df_unemployed.columns[1:12]]

In [34]:
df_unem.head()

,Kreis_ID,Kreis/Stadt,Arbeitslose Anzahl insgesamt,Arbeitslose Anzahl Ausländer,Arbeitslose Anzahl Schwerbehindert,Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre,Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre,Arbeitslose Anzahl Alter zwischen 55 bis unter 65 Jahre,Arbeitslose Anzahl Langzeitarbeitslos,Arbeitslosenquote Prozent bezogen auf abhängige zivile Erwerbspersonen,Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen
0,DG,Deutschland,2695444,795692,169691,49477,257361,562573,816749,"6,5","5,9"
1,01,Schleswig-Holstein,92140,21506,5197,1842,9308,19359,28170,"6,5","5,8"
2,01001,"Flensburg, kreisfreie Stadt",4722,1144,274,102,568,782,1419,"9,9","9,1"
3,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",11776,3415,535,183,931,1959,4159,"9,3","8,5"
4,01003,"Lübeck, Hansestadt, kreisfreie Stadt",9404,2315,477,190,922,1746,3131,"9,0","8,1"


In [35]:
# Entfernen der vorangestellten 0 bei den vierstelligen Kreis_IDs, da für den Join nicht benötigt

for col in df_unem:
    df_unem['Kreis_ID'] = df_unem['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)

df_unem.head()

C:\Users\Andreas\AppData\Local\Temp/ipykernel_28820/2422749098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unem['Kreis_ID'] = df_unem['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)


,Kreis_ID,Kreis/Stadt,Arbeitslose Anzahl insgesamt,Arbeitslose Anzahl Ausländer,Arbeitslose Anzahl Schwerbehindert,Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre,Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre,Arbeitslose Anzahl Alter zwischen 55 bis unter 65 Jahre,Arbeitslose Anzahl Langzeitarbeitslos,Arbeitslosenquote Prozent bezogen auf abhängige zivile Erwerbspersonen,Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen
0,DG,Deutschland,2695444,795692,169691,49477,257361,562573,816749,"6,5","5,9"
1,1,Schleswig-Holstein,92140,21506,5197,1842,9308,19359,28170,"6,5","5,8"
2,1001,"Flensburg, kreisfreie Stadt",4722,1144,274,102,568,782,1419,"9,9","9,1"
3,1002,"Kiel, Landeshauptstadt, kreisfreie Stadt",11776,3415,535,183,931,1959,4159,"9,3","8,5"
4,1003,"Lübeck, Hansestadt, kreisfreie Stadt",9404,2315,477,190,922,1746,3131,"9,0","8,1"


In [36]:
# Bearbeitung von Hamburg und Berlin, um Join zu ermöglichen
# Spalte Kreis/Stadt löschen

df_unem.loc[df_unem['Kreis_ID']=='11','Kreis_ID']='11000'
df_unem.loc[df_unem['Kreis_ID']=='2','Kreis_ID']='2000'

#df_unem[df_unem["Kreis/Stadt"].str.contains("Hamburg")==True]
#df_unem[df_unem["Kreis/Stadt"].str.contains("Berlin")==True]

df_unem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column                                                                  Non-Null Count  Dtype 
---  ------                                                                  --------------  ----- 
 0   Kreis_ID                                                                542 non-null    object
 1   Kreis/Stadt                                                             542 non-null    object
 2   Arbeitslose Anzahl insgesamt                                            542 non-null    object
 3   Arbeitslose Anzahl Ausländer                                            542 non-null    object
 4   Arbeitslose Anzahl Schwerbehindert                                      542 non-null    object
 5   Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre                 542 non-null    object
 6   Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre                 542 non-null    ob

C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [37]:
# Auswahl aller Kreis_IDs mit Zeilenlänge > 4
# Umwandelung in Datentyp int und float

df_unem=df_unem[df_unem.apply(lambda x: len(x['Kreis_ID'])>=4, axis=1)]

df_unem=df_unem.replace('.',None)
df_unem=df_unem.replace('-',None)
df_unem=df_unem.replace('x',None)
df_unem=df_unem.replace(',','.', regex=True)
df_unem=df_unem.drop(['Kreis/Stadt'], axis=1)

df_unem[df_unem.columns[0:7]]=df_unem.iloc[:,0:7].apply(pd.to_numeric)
df_unem[df_unem.columns[8:9]]=df_unem.iloc[:,8:9].astype(float, errors = 'raise')

df_unem.head()

,Kreis_ID,Arbeitslose Anzahl insgesamt,Arbeitslose Anzahl Ausländer,Arbeitslose Anzahl Schwerbehindert,Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre,Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre,Arbeitslose Anzahl Alter zwischen 55 bis unter 65 Jahre,Arbeitslose Anzahl Langzeitarbeitslos,Arbeitslosenquote Prozent bezogen auf abhängige zivile Erwerbspersonen,Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen
2,1001,4722,1144,274,102,568,782,1419,9.9,9.1
3,1002,11776,3415,535,183,931,1959,4159,9.3,8.5
4,1003,9404,2315,477,190,922,1746,3131,9.0,8.1
5,1004,3846,916,211,92,441,683,1334,9.9,8.9
6,1051,4261,736,228,123,567,885,1188,6.8,6.1


In [38]:
df_unem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 2 to 537
Data columns (total 10 columns):
 #   Column                                                                  Non-Null Count  Dtype  
---  ------                                                                  --------------  -----  
 0   Kreis_ID                                                                488 non-null    int64  
 1   Arbeitslose Anzahl insgesamt                                            488 non-null    int64  
 2   Arbeitslose Anzahl Ausländer                                            488 non-null    int64  
 3   Arbeitslose Anzahl Schwerbehindert                                      488 non-null    int64  
 4   Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre                 488 non-null    int64  
 5   Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre                 488 non-null    int64  
 6   Arbeitslose Anzahl Alter zwischen 55 bis unter 65 Jahre                 488 non-nu

In [39]:
# Join an den Kriminaldatensatz

group_criminal2=pd.merge(group_criminal1, df_unem, on=['Kreis_ID'],how='left')
group_criminal2.head()

,Kreis_ID,Straftat_Cluster,Stadt-/Landkreis,Anzahl erfasste Faelle,Haeufigkeitszahl,"erfasste Faelle, davon Versuche - Anzahl",mit Schusswaffe gedroht,mit Schusswaffe geschossen,Auffklaerung - Anzahl Faelle,Tatverdaechtige insgesamt,...,Allg. Absolventen Anzahl mit schulischen Teil der Fachhochschulreife Insgesamt,Arbeitslose Anzahl insgesamt,Arbeitslose Anzahl Ausländer,Arbeitslose Anzahl Schwerbehindert,Arbeitslose Anzahl Alter zwischen 15 bis unter 20 Jahre,Arbeitslose Anzahl Alter zwischen 15 bis unter 25 Jahre,Arbeitslose Anzahl Alter zwischen 55 bis unter 65 Jahre,Arbeitslose Anzahl Langzeitarbeitslos,Arbeitslosenquote Prozent bezogen auf abhängige zivile Erwerbspersonen,Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen
0,1001,Beförderungserschleichung,Flensburg,57,63,0,0,0,56,56,...,60,4722,1144,274,102,568,782,1419,9.9,9.1
1,1001,"Begünstigung, Strafvereitelung, Hehlerei, Geld...",Flensburg,25,28,0,0,0,24,27,...,60,4722,1144,274,102,568,782,1419,9.9,9.1
2,1001,Betrug,Flensburg,797,884,74,0,0,585,442,...,60,4722,1144,274,102,568,782,1419,9.9,9.1
3,1001,Brandstiftung,Flensburg,28,31,4,0,0,12,12,...,60,4722,1144,274,102,568,782,1419,9.9,9.1
4,1001,Cyberkriminalität,Flensburg,172,191,36,0,0,84,60,...,60,4722,1144,274,102,568,782,1419,9.9,9.1


In [40]:
#Check auf missing values

group_criminal2.isnull().sum(axis=0)

Kreis_ID                                                                                          0
Straftat_Cluster                                                                                  0
Stadt-/Landkreis                                                                                  0
Anzahl erfasste Faelle                                                                            0
Haeufigkeitszahl                                                                                  0
erfasste Faelle, davon Versuche - Anzahl                                                          0
mit Schusswaffe gedroht                                                                           0
mit Schusswaffe geschossen                                                                        0
Auffklaerung - Anzahl Faelle                                                                      0
Tatverdaechtige insgesamt                                                                         0


#### Import von Daten zur Bundestagswahl 2021

Der Datensatz enthält Daten zur Bundestagswahl im September 2021. Konkret werden die Anzahl der Wahlberechtigte, Wahlbeteiligung sowie die Anzahl gültiger Zweitstimmen nach ausgewählten Parteien. 

In [41]:
# Import der Daten aus dem github-repository

url_election = 'https://raw.githubusercontent.com/andreashrb/Python_for_DS_Project/main/data/Bundestagswahl_2021_Ergebnisse.csv'

df_election = pd.read_csv(url_election,encoding="latin-1",skiprows=4,delimiter=";", decimal=",",header=[1,2,3,4])

df_election.replace('-', None)

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                    Unnamed: 0_level_1 Unnamed: 1_level_1   
                                    Unnamed: 0_level_2 Unnamed: 1_level_2   
                                    Unnamed: 0_level_3 Unnamed: 1_level_3   
0                                           26.09.2021                 DG   
1                                           26.09.2021                 01   
2                                           26.09.2021              01001   
3                                           26.09.2021              01002   
4                                           26.09.2021              01003   
..                                                 ...                ...   
537                                         26.09.2021              16077   
538                                         __________                NaN   
539  zu "gesamte Tabelle":\nBaden-Württemberg (Beri...                NaN   
540  © Statistische Ämter des Bundes und der Länder...                NaN   
541                       Stand: 11.02.2022 / 20:53:28                NaN   

                                 Unnamed: 2_level_0    Wahlberechtigte  \
                                 Unnamed: 2_level_1 Unnamed: 3_level_1   
                                 Unnamed: 2_level_2 Unnamed: 3_level_2   
                                 Unnamed: 2_level_3             Anzahl   
0                                       Deutschland           61181072   
1                                Schleswig-Holstein            2272717   
2                       Flensburg, kreisfreie Stadt              69302   
3          Kiel, Landeshauptstadt, kreisfreie Stadt             184655   
4              Lübeck, Hansestadt, kreisfreie Stadt             164139   
..                                              ...                ...   
537                         Altenburger Land, Kreis              74029   
538                                             NaN                NaN   
539                                             NaN                NaN   
540                                             NaN                NaN   
541                                             NaN                NaN   

       Wahlbeteiligung Gültige Zweitstimmen                               \
    Unnamed: 4_level_1   Unnamed: 5_level_1  Parteien                      
    Unnamed: 4_level_2   Unnamed: 5_level_2   CDU/CSU       SPD    GRÜNE   
               Prozent               Anzahl    Anzahl    Anzahl   Anzahl   
0                 76,6             46442023  11178298  11955434  6852206   
1                 78,2              1762754    388399    494055   322763   
2                 74,0                50818      7152     12480    12296   
3                 76,2               139961     20333     36407    40476   
4                 72,4               117674     20757     36136    27112   
..                 ...                  ...       ...       ...      ...   
537               72,4                52844      9085     11577     2064   
538                NaN                  NaN       NaN       NaN      NaN   
539                NaN                  NaN       NaN       NaN      NaN   
540                NaN                  NaN       NaN       NaN      NaN   
541                NaN                  NaN       NaN       NaN      NaN   

                                                   
                                                   
         FDP DIE LINKE      AfD Sonstige Parteien  
      Anzahl    Anzahl   Anzahl            Anzahl  
0    5319952   2270906  4803902           4061325  
1     220039     64238   119566            153694  
2       4780      3153     2756              8201  
3      14335      8978     7023             12409  
4      11963      5360     7807              8539  
..       ...       ...      ...               ...  
537     5263      5194    15576              4085  
538      NaN       NaN      NaN               NaN  
539 

In [42]:
# Auflösen und Zusammenführen der mehrstufigen Spaltenstruktur sowie Umbenennung der Spalten

df_election.columns = [' '.join(col) for col in df_election]

df_election.rename(columns={'Unnamed: 1_level_0 Unnamed: 1_level_1 Unnamed: 1_level_2 Unnamed: 1_level_3':'Kreis_ID',
                                        'Unnamed: 2_level_0 Unnamed: 2_level_1 Unnamed: 2_level_2 Unnamed: 2_level_3':'Kreis/Stadt',
                                        'Wahlberechtigte Unnamed: 3_level_1 Unnamed: 3_level_2 Anzahl':'Wahlberechtige Anzahl',
                                        'Wahlbeteiligung Unnamed: 4_level_1 Unnamed: 4_level_2 Prozent':'Wahlbeteiligung Prozent',
                                        'Gültige Zweitstimmen Unnamed: 5_level_1 Unnamed: 5_level_2 Anzahl':'Gültige Zeitstimmen Anzahl'
                                        },inplace=True)

df_elec=df_election[df_election.columns[1:12]]

In [43]:
df_elec.head()

,Kreis_ID,Kreis/Stadt,Wahlberechtige Anzahl,Wahlbeteiligung Prozent,Gültige Zeitstimmen Anzahl,Gültige Zweitstimmen Parteien CDU/CSU Anzahl,Gültige Zweitstimmen Parteien SPD Anzahl,Gültige Zweitstimmen Parteien GRÜNE Anzahl,Gültige Zweitstimmen Parteien FDP Anzahl,Gültige Zweitstimmen Parteien DIE LINKE Anzahl,Gültige Zweitstimmen Parteien AfD Anzahl
0,DG,Deutschland,61181072,"76,6",46442023,11178298,11955434,6852206,5319952,2270906,4803902
1,01,Schleswig-Holstein,2272717,"78,2",1762754,388399,494055,322763,220039,64238,119566
2,01001,"Flensburg, kreisfreie Stadt",69302,"74,0",50818,7152,12480,12296,4780,3153,2756
3,01002,"Kiel, Landeshauptstadt, kreisfreie Stadt",184655,"76,2",139961,20333,36407,40476,14335,8978,7023
4,01003,"Lübeck, Hansestadt, kreisfreie Stadt",164139,"72,4",117674,20757,36136,27112,11963,5360,7807


In [44]:
# Entfernen der vorangestellten 0 bei den vierstelligen Kreis_IDs, da für den Join nicht benötigt

for col in df_elec:
    df_elec['Kreis_ID'] = df_elec['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)

df_elec.head()

C:\Users\Andreas\AppData\Local\Temp/ipykernel_28820/2689545011.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elec['Kreis_ID'] = df_elec['Kreis_ID'].apply(str).apply(lambda x : x[1:] if x.startswith('0') else x)


,Kreis_ID,Kreis/Stadt,Wahlberechtige Anzahl,Wahlbeteiligung Prozent,Gültige Zeitstimmen Anzahl,Gültige Zweitstimmen Parteien CDU/CSU Anzahl,Gültige Zweitstimmen Parteien SPD Anzahl,Gültige Zweitstimmen Parteien GRÜNE Anzahl,Gültige Zweitstimmen Parteien FDP Anzahl,Gültige Zweitstimmen Parteien DIE LINKE Anzahl,Gültige Zweitstimmen Parteien AfD Anzahl
0,DG,Deutschland,61181072,"76,6",46442023,11178298,11955434,6852206,5319952,2270906,4803902
1,1,Schleswig-Holstein,2272717,"78,2",1762754,388399,494055,322763,220039,64238,119566
2,1001,"Flensburg, kreisfreie Stadt",69302,"74,0",50818,7152,12480,12296,4780,3153,2756
3,1002,"Kiel, Landeshauptstadt, kreisfreie Stadt",184655,"76,2",139961,20333,36407,40476,14335,8978,7023
4,1003,"Lübeck, Hansestadt, kreisfreie Stadt",164139,"72,4",117674,20757,36136,27112,11963,5360,7807


In [45]:
# Bearbeitung von Hamburg und Berlin, um Join zu ermöglichen
# Spalte Kreis/Stadt löschen

df_elec.loc[df_elec['Kreis_ID']=='11','Kreis_ID']='11000'
df_elec.loc[df_elec['Kreis_ID']=='2','Kreis_ID']='2000'

df_elec[df_elec["Kreis/Stadt"].str.contains("Hamburg")==True]
df_elec[df_elec["Kreis/Stadt"].str.contains("Berlin")==True]

df_elec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   Kreis_ID                                        542 non-null    object
 1   Kreis/Stadt                                     538 non-null    object
 2   Wahlberechtige Anzahl                           538 non-null    object
 3   Wahlbeteiligung Prozent                         538 non-null    object
 4   Gültige Zeitstimmen Anzahl                      538 non-null    object
 5   Gültige Zweitstimmen Parteien CDU/CSU Anzahl    538 non-null    object
 6   Gültige Zweitstimmen Parteien SPD Anzahl        538 non-null    object
 7   Gültige Zweitstimmen Parteien GRÜNE Anzahl      538 non-null    object
 8   Gültige Zweitstimmen Parteien FDP Anzahl        538 non-null    object
 9   Gültige Zweitstimmen Parteien DIE LINKE Anzahl  538 no

C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Andreas\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [46]:
# Auswahl aller Kreis_IDs mit Zeilenlänge > 4
# Umwandelung in Datentyp int und float

df_elec=df_elec.replace('.',None)
df_elec=df_elec.replace('-',None)
df_elec=df_elec.replace('x',None)
df_elec=df_elec.replace(',','.', regex=True)
df_elec=df_elec.drop(['Kreis/Stadt'], axis=1)

df_elec=df_elec[df_elec.apply(lambda x: len(x['Kreis_ID'])>=4, axis=1)]

df_elec[df_elec.columns[0:2]]=df_elec.iloc[:,0:2].astype('float').astype('Int64')
df_elec[df_elec.columns[2]]=df_elec.iloc[:,2].astype('float')
df_elec[df_elec.columns[3:10]]=df_elec.iloc[:,3:10].astype('float').astype('Int64')

df_elec.head()

,Kreis_ID,Wahlberechtige Anzahl,Wahlbeteiligung Prozent,Gültige Zeitstimmen Anzahl,Gültige Zweitstimmen Parteien CDU/CSU Anzahl,Gültige Zweitstimmen Parteien SPD Anzahl,Gültige Zweitstimmen Parteien GRÜNE Anzahl,Gültige Zweitstimmen Parteien FDP Anzahl,Gültige Zweitstimmen Parteien DIE LINKE Anzahl,Gültige Zweitstimmen Parteien AfD Anzahl
2,1001,69302,74.0,50818,7152,12480,12296,4780,3153,2756
3,1002,184655,76.2,139961,20333,36407,40476,14335,8978,7023
4,1003,164139,72.4,117674,20757,36136,27112,11963,5360,7807
5,1004,59460,67.9,39971,8089,12872,5841,4762,1320,3751
6,1051,107187,76.0,80616,20545,22529,9686,11464,2545,7098


In [47]:
df_elec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 488 entries, 2 to 537
Data columns (total 10 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Kreis_ID                                        488 non-null    Int64  
 1   Wahlberechtige Anzahl                           488 non-null    Int64  
 2   Wahlbeteiligung Prozent                         488 non-null    float64
 3   Gültige Zeitstimmen Anzahl                      488 non-null    Int64  
 4   Gültige Zweitstimmen Parteien CDU/CSU Anzahl    488 non-null    Int64  
 5   Gültige Zweitstimmen Parteien SPD Anzahl        488 non-null    Int64  
 6   Gültige Zweitstimmen Parteien GRÜNE Anzahl      488 non-null    Int64  
 7   Gültige Zweitstimmen Parteien FDP Anzahl        488 non-null    Int64  
 8   Gültige Zweitstimmen Parteien DIE LINKE Anzahl  488 non-null    Int64  
 9   Gültige Zweitstimmen Parteien AfD Anzahl   

In [48]:
# Join an den Kriminaldatensatz

group_criminal3=pd.merge(group_criminal2, df_elec, on=['Kreis_ID'],how='left')
group_criminal3.head()

,Kreis_ID,Straftat_Cluster,Stadt-/Landkreis,Anzahl erfasste Faelle,Haeufigkeitszahl,"erfasste Faelle, davon Versuche - Anzahl",mit Schusswaffe gedroht,mit Schusswaffe geschossen,Auffklaerung - Anzahl Faelle,Tatverdaechtige insgesamt,...,Arbeitslosenquote Prozent bezogen auf alle zivilen Erwerbspersonen,Wahlberechtige Anzahl,Wahlbeteiligung Prozent,Gültige Zeitstimmen Anzahl,Gültige Zweitstimmen Parteien CDU/CSU Anzahl,Gültige Zweitstimmen Parteien SPD Anzahl,Gültige Zweitstimmen Parteien GRÜNE Anzahl,Gültige Zweitstimmen Parteien FDP Anzahl,Gültige Zweitstimmen Parteien DIE LINKE Anzahl,Gültige Zweitstimmen Parteien AfD Anzahl
0,1001,Beförderungserschleichung,Flensburg,57,63,0,0,0,56,56,...,9.1,69302,74.0,50818,7152,12480,12296,4780,3153,2756
1,1001,"Begünstigung, Strafvereitelung, Hehlerei, Geld...",Flensburg,25,28,0,0,0,24,27,...,9.1,69302,74.0,50818,7152,12480,12296,4780,3153,2756
2,1001,Betrug,Flensburg,797,884,74,0,0,585,442,...,9.1,69302,74.0,50818,7152,12480,12296,4780,3153,2756
3,1001,Brandstiftung,Flensburg,28,31,4,0,0,12,12,...,9.1,69302,74.0,50818,7152,12480,12296,4780,3153,2756
4,1001,Cyberkriminalität,Flensburg,172,191,36,0,0,84,60,...,9.1,69302,74.0,50818,7152,12480,12296,4780,3153,2756


In [49]:
#Check auf missing values

group_criminal3.isnull().sum(axis=0)

Kreis_ID                                                                                          0
Straftat_Cluster                                                                                  0
Stadt-/Landkreis                                                                                  0
Anzahl erfasste Faelle                                                                            0
Haeufigkeitszahl                                                                                  0
erfasste Faelle, davon Versuche - Anzahl                                                          0
mit Schusswaffe gedroht                                                                           0
mit Schusswaffe geschossen                                                                        0
Auffklaerung - Anzahl Faelle                                                                      0
Tatverdaechtige insgesamt                                                                         0


#### Export als csv

In [50]:
group_criminal3.to_csv("DataPreparation_Andreas.csv",sep=",")

In [51]:
group_criminal3.shape[0]

8822